In [2]:
import torch
import numpy
import torch.nn as nn
import torch.nn.functional as F

In [3]:
##word embedding
##source sentence & targert sentence

batch_size = 2
##单词表大小
max_num_src_words = 8
max_num_tgt_words = 8

##最大序列长度
max_arc_seq_len = 5
max_tgt_seq_len = 5

#src_len = torch.randint(2, 5, (batch_size,))
#tgt_len = torch.randint(2, 5, (batch_size,))
##两个句子，第一个长度2，第二个长度4
src_len = torch.Tensor([2,4]).to(torch.int32)
tgt_len = torch.Tensor([4,3]).to(torch.int32)

##单词索引序列
##pad填充序列为统一最大长度，默认填充0
##变为二维tensor
src_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1, max_num_src_words, (L,)),(0, max_arc_seq_len-L)),0)
                                      for L in src_len])
tgt_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1, max_num_tgt_words, (L,)),(0, max_tgt_seq_len-L)), 0)
                                      for L in src_len])
print(src_seq)
print(tgt_seq)

tensor([[1, 4, 0, 0, 0],
        [6, 6, 5, 2, 0]])
tensor([[7, 6, 0, 0, 0],
        [4, 4, 2, 7, 0]])


In [4]:
##word embedding
##embeding向量长度8
model_dim = 8
src_embedding_table = nn.Embedding(max_num_src_words,model_dim)
tgt_embedding_table = nn.Embedding(max_num_tgt_words,model_dim)
src_embedding = src_embedding_table(src_seq)
tgt_embedding = tgt_embedding_table(tgt_seq)
print(src_embedding_table.weight)
print(src_seq)
print(src_embedding)

Parameter containing:
tensor([[ 0.2150, -0.2147, -0.6253, -0.3334, -1.0708, -1.6920, -1.1532, -0.3522],
        [ 0.8549, -1.4145,  1.5203,  2.0198, -0.2876, -0.4578,  0.8444, -0.1299],
        [ 0.9728,  1.7071,  1.9642,  0.0505, -2.1196,  1.0147,  0.4731,  0.7802],
        [-1.4867,  0.4381,  1.1285, -0.9793,  0.2671,  0.5633,  0.0424,  0.3817],
        [ 0.3047,  0.9499,  1.8239,  2.4549, -0.0083, -1.4732, -0.2489,  1.0262],
        [ 0.4302,  0.1872, -0.0829, -1.4862, -0.8040,  0.8736, -1.5349, -1.1522],
        [-0.3900, -1.6287,  0.2112, -0.2127,  0.2220,  0.1421,  2.6023,  0.0493],
        [ 1.5544,  1.6540, -2.3084, -0.9239, -0.9263,  0.7701, -0.6956,  0.0987]],
       requires_grad=True)
tensor([[1, 4, 0, 0, 0],
        [6, 6, 5, 2, 0]])
tensor([[[ 0.8549, -1.4145,  1.5203,  2.0198, -0.2876, -0.4578,  0.8444,
          -0.1299],
         [ 0.3047,  0.9499,  1.8239,  2.4549, -0.0083, -1.4732, -0.2489,
           1.0262],
         [ 0.2150, -0.2147, -0.6253, -0.3334, -1.0708, -1

In [5]:
##position embedding
max_position_len = 5
pos_mat = torch.arange(max_position_len).reshape((-1,1))
i_mat = torch.pow(10000,torch.arange(0, 8, 2).reshape(1,-1)/model_dim)
pe_embedding_table = torch.zeros(max_position_len, model_dim)
pe_embedding_table[:, 0::2] = torch.sin(pos_mat/i_mat)
pe_embedding_table[:, 1::2] = torch.cos(pos_mat/i_mat)
#print(pe_embedding_table)

pe_embedding = nn.Embedding(max_position_len, model_dim)
pe_embedding.weight = nn.Parameter(pe_embedding_table, requires_grad=False)
#print(pe_embedding.weight)
src_pos = torch.cat([torch.unsqueeze(torch.arange(max(src_len)),0) for _ in src_len]).to(torch.int32)
print(src_pos)
tgt_pos = torch.cat([torch.unsqueeze(torch.arange(max(tgt_len)),0) for _ in tgt_len]).to(torch.int32)

src_pe_embedding = pe_embedding(src_pos)
tgt_pe_embedding = pe_embedding(tgt_pos)
print(src_pe_embedding)

tensor([[0, 1, 2, 3],
        [0, 1, 2, 3]], dtype=torch.int32)
tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           1.0000e+00,  0.0000e+00,  1.0000e+00],
         [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
           9.9995e-01,  1.0000e-03,  1.0000e+00],
         [ 9.0930e-01, -4.1615e-01,  1.9867e-01,  9.8007e-01,  1.9999e-02,
           9.9980e-01,  2.0000e-03,  1.0000e+00],
         [ 1.4112e-01, -9.8999e-01,  2.9552e-01,  9.5534e-01,  2.9995e-02,
           9.9955e-01,  3.0000e-03,  1.0000e+00]],

        [[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           1.0000e+00,  0.0000e+00,  1.0000e+00],
         [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
           9.9995e-01,  1.0000e-03,  1.0000e+00],
         [ 9.0930e-01, -4.1615e-01,  1.9867e-01,  9.8007e-01,  1.9999e-02,
           9.9980e-01,  2.0000e-03,  1.0000e+00],
         [ 1.4112e-01, -9.8999e-01,  2.9552e-01,  9.5534e-0

In [29]:
##mask的shape:[batch_size,max_scr_len,max_scr_len],值为inf或1
##先有效位再padding
##batch为2扩一维Z
valid_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(src_len)-L)),0) \
                                               for L in src_len]), 2)
#print(valid_encoder_pos.shape)
##三维矩阵相乘
valid_encorder_pos_matrix = torch.bmm(valid_encoder_pos, valid_encoder_pos.transpose(1,2))
##相当于邻接矩阵
#print(valid_encorder_pos_matrix)
#print(src_len)

invalid_encorder_pos_matrix = 1 - valid_encorder_pos_matrix
#print(invalid_encorder_pos_matrix)

##true就mask
mask_encoder_self_attention = invalid_encorder_pos_matrix.to(torch.bool)
#print(mask_encoder_self_attention)

score = torch .randn(batch_size, max(src_len), max(src_len))
##为0部分fill -1e9
masked_score = score.masked_fill(mask_encoder_self_attention, -1e9)
prob = F.softmax(masked_score, -1)

#print(score)
#print(src_len)
print(masked_score)
print(prob)

tensor([[[ 6.6549e-01,  1.4553e-02, -1.0000e+09, -1.0000e+09],
         [-1.7719e+00, -3.5355e-01, -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09]],

        [[ 1.2813e-01,  1.8046e-01, -1.3900e-01,  1.8591e+00],
         [-8.6575e-01, -1.0751e+00, -1.1181e+00,  7.3859e-02],
         [-1.3175e+00, -6.0572e-02, -9.1582e-01, -3.3958e+00],
         [-4.0506e-01, -1.0690e+00, -9.1746e-01,  8.7240e-01]]])
tensor([[[0.6572, 0.3428, 0.0000, 0.0000],
         [0.1949, 0.8051, 0.0000, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500],
         [0.2500, 0.2500, 0.2500, 0.2500]],

        [[0.1181, 0.1245, 0.0904, 0.6670],
         [0.1943, 0.1576, 0.1509, 0.4972],
         [0.1630, 0.5730, 0.2436, 0.0204],
         [0.1754, 0.0903, 0.1051, 0.6292]]])


In [35]:
#intra-attention的mask
##Q @ K^T shape:[batch,tgt_seq_len, src_seq_len]
valid_decoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L), (0, max(tgt_len)-L)),0) \
                                               for L in tgt_len]), 2)
##scr与tgt有效性
valid_cross_pos_matrix = torch.bmm(valid_decoder_pos, valid_encoder_pos.transpose(1,2))
#print(valid_encoder_pos)
#print(valid_decoder_pos)
#print(valid_cross_pos)
invalid_cross_pos_matrix = 1 - valid_cross_pos_matrix
mask_cross_attention = invalid_cross_pos_matrix.to(torch.bool)
#print(mask_cross_attention)

tensor([[[False, False,  True,  True],
         [False, False,  True,  True],
         [False, False,  True,  True],
         [False, False,  True,  True]],

        [[False, False, False, False],
         [False, False, False, False],
         [False, False, False, False],
         [ True,  True,  True,  True]]])


In [46]:
##decorder self-attention的mask(三角)
valid_decoder_tri_matrix = torch.cat([torch.unsqueeze(F.pad(torch.tril(torch.ones(L,L)), \
                                                            (0,max(tgt_len)-L,0,max(tgt_len)-L)),0) \
                                      for L in tgt_len])
#print(valid_decoder_tri_matrix)
#print(valid_decoder_tri_matrix.shape)
invalid_decoder_tri_matrix = 1-valid_decoder_tri_matrix
invalid_decoder_tri_matrix = invalid_decoder_tri_matrix.to(torch.bool)
#print(invalid_decoder_tri_matrix)
score = torch .randn(batch_size, max(tgt_len), max(tgt_len))
##为0部分fill -1e9
masked_score = score.masked_fill(invalid_decoder_tri_matrix, -1e9)
prob = F.softmax(masked_score, -1)
#print(prob)

tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.6193, 0.3807, 0.0000, 0.0000],
         [0.4531, 0.2043, 0.3426, 0.0000],
         [0.1401, 0.1951, 0.2347, 0.4301]],

        [[1.0000, 0.0000, 0.0000, 0.0000],
         [0.1941, 0.8059, 0.0000, 0.0000],
         [0.1540, 0.5431, 0.3029, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500]]])


In [48]:
##self-attention
##shape of Q,K,V : (batch_size*num_head, seq_len,model_dim/num_head)
def scaled_dot_product_attention(Q,K,V,attn_mask):
    score = torch.bmm(Q,K,transpose(-2,-1))/torch.sqrt(model_dim)
    masked_score = score.masked_fill(attn_mask, -1e9)
    prob = F.softmax(masked_score, -1)
    context = torch.bmm(prob,V)
    return context